<a href="https://colab.research.google.com/github/Mario-td/HandGestureClassification/blob/master/GatheringData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Gathering data from text files**

In [1]:
import pandas as pd
import numpy as np

In [ ]:
# Allows access to Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [3]:
# Functions for loading the data and labels
def load_data(path, steps):
    file = open(path, 'r')
    data = []
    for line in file.readlines():
        data.append([float(i) for i in line.split(',') if i.strip()]) 
    file.close()

    # Subdivide the list by the number of steps per sample
    data = [data[x:x+steps] for x in range(0, len(data), steps)]
    return data 

def load_label(path):
    file = open(path, 'r')
    label = []
    for line in file.readlines():
        for i in line.split():
            l = int(i) - 1 # -1 for indexing from 0
        label.append(l) 
    file.close()
    return label 

# Function for naming the columns
def feat3D(n_features3D):
    feat = []
    for i in range(0, n_features3D):
        feat.append('j%d_x' % i)
        feat.append('j%d_y' % i)
        feat.append('j%d_z' % i)
    return feat

In [4]:
# Number of steps per sample
n_steps = 32
# Number of keypoint per step
n_keypoints = 21;

# Path to the files
path = '/content/drive/My Drive/MastersThesis/Dataset/'

# Loads data and labels
X_3D = load_data(path + 'X_3D.txt', n_steps)
y_ = load_label(path + 'Y_.txt')

In [5]:
# Visualizes the shape of the arrays
print(np.array(X_3D).shape)
print(np.array(y_).shape)

(3521, 32, 63)
(3521,)


# **Structures the dataset**

In [6]:
# Sets the labels as index and the coordinate arrays as columns
df = pd.DataFrame(X_3D, index = y_)
# MultiIndex with labels and sequences
df = df.apply(pd.Series).stack()
df = pd.DataFrame(df, columns = ['3D position'])
df = df.rename_axis(['Label','Sequence'])
df

3D position
Label Sequence                                                   
0     0         [273.65625, 375.640625, 0.0, 312.046875, 326.2...
      1         [241.234375, 374.203125, 0.0, 284.609375, 319....
      2         [233.46875, 369.3125, 0.0, 266.28125, 320.0937...
      3         [223.5, 358.78125, 0.0, 256.03125, 315.40625, ...
      4         [219.0, 360.0, 0.0, 252.0, 310.5, 257.0, 274.0...
...                                                           ...
4     27        [378.09375, 274.6875, 2570.0, 418.59375, 256.9...
      28        [380.21875, 291.375, 2570.0, 414.671875, 264.5...
      29        [375.53125, 289.046875, 2570.0, 410.15625, 263...
      30        [379.3125, 288.90625, 2313.0, 409.71875, 267.1...
      31        [375.0, 286.75, 2313.0, 409.0, 261.25, 2313.0,...

[112672 rows x 1 columns]

In [7]:
# Names the new columns and splits them
df[feat3D(n_keypoints)] = pd.DataFrame(df['3D position'].tolist(), index= df.index)
del df['3D position'] # Delete the first column
df

j0_x        j0_y    j0_z  ...       j20_x       j20_y   j20_z
Label Sequence                                  ...                                
0     0         273.656250  375.640625     0.0  ...  196.875000  200.140625     0.0
      1         241.234375  374.203125     0.0  ...  132.796875  211.546875     0.0
      2         233.468750  369.312500     0.0  ...  113.156250  210.718750     0.0
      3         223.500000  358.781250     0.0  ...  104.218750  217.812500     0.0
      4         219.000000  360.000000     0.0  ...   87.000000  217.000000     0.0
...                    ...         ...     ...  ...         ...         ...     ...
4     27        378.093750  274.687500  2570.0  ...  373.031250  201.281250  2570.0
      28        380.218750  291.375000  2570.0  ...  334.281250  153.562500  2570.0
      29        375.531250  289.046875  2570.0  ...  306.281250  163.531250  2313.0
      30        379.312500  288.906250  2313.0  ...  301.125000  167.281250  2570.0
      31        375.000000  286.750000  2313.0  ...  294.250000  172.000000  2570.0

[112672 rows x 63 columns]

In [8]:
# Saves the dataset in CSV format
df.to_csv(path + '/HandGesture3D.csv')